In [1]:
class Permutation(tuple):
    def __new__(cls, elements):
        '''
        >>> Permutation('53412')
        (5, 3, 4, 1, 2)
        >>> Permutation([5,3,4,1,2])
        (5, 3, 4, 1, 2)
        >>> Permutation(('5','3','4','1','2'))
        (5, 3, 4, 1, 2)
        '''
        return tuple.__new__(cls, map(int, elements))
    
    def __call__(self, i):
        '''
        >>> w = Permutation([5,2,4,1,3])
        >>> w(3)
        4
        '''
        return self[i - 1]
    
    
    def __mul__(self, other):
        '''
        >>> one = Permutation('32541')
        >>> two = Permutation('25134')
        >>> one * two
        (2, 1, 3, 5, 4)
        '''
        if (len(self) != len(other)):
            raise ValueError('permutations should have the same size')
        return Permutation([self(other(i)) for i in range(1, len(self) + 1)])
    
    def __rmul__(self, other):
        '''
        >>> one = '32541'
        >>> two = Permutation('25134')
        >>> one * two
        (2, 1, 3, 5, 4)
        '''
        return Permutation(other) * self

    def code(self):
        '''
        >>> w = Permutation([5,2,4,1,3])
        >>> w.code()
        [4, 1, 2, 0, 0]
        '''
        result = []
        for i in range(1, len(self) + 1):
            count = 0
            for j in range(i, len(self) + 1):
                if self(i) > self(j):
                    count += 1
            result.append(count)
        return result
    
    def des_num(self):
        '''
        >>> w = Permutation([5,2,4,1,3])
        >>> w.des_num()
        2
        '''
        return len(self.descents())
    
    def descents(self):
        '''
        >>> w = Permutation([5,2,4,1,3])
        >>> w.descents()
        [1, 3]
        '''
        return [i for i in range(1, len(self)) if self(i) > self(i + 1)]
    
    def maj(self):
        '''
        >>> w = Permutation([5,2,4,1,3])
        >>> w.maj()
        4
        '''
        return sum(self.descents())
    
    def imaj(self):
        '''
        >>> w = Permutation([5,2,4,1,3])
        >>> w.imaj()
        8
        '''
        return self.inverse().maj()
    
    def inverse(self):
        '''
        >>> w = Permutation([5,2,4,1,3])
        >>> w.inverse()
        (4, 2, 5, 3, 1)
        >>> w.inverse() * w
        (1, 2, 3, 4, 5)
        '''
        res = [0] * len(self)
        for i in range(len(self)):
            res[self[i] - 1] = i + 1
        return Permutation(res)

    def cycles(self):
        '''
        >>> w = Permutation([5,2,4,1,3])
        >>> w.cycles()
        [[1, 5, 3, 4], [2]]
        '''
        result = []
        def aux(perm, index, ctx):
            ctx.append(index)
            if perm(index) in ctx:
                return ctx
            return aux(perm, perm(index), ctx)
        already = []
        for i in range(1, len(self) + 1):
            if i not in already:
                cycle = aux(self, i, [])
                already.extend(cycle)
                result.append(cycle)
        return result
    
    def reduced_dec(self):
        '''
        >>> w = Permutation([5,2,4,1,3])
        >>> w.reduced_dec()
        [3, 4, 1, 2, 3, 2, 1]
        '''
        sort = True
        first = True
        ll = list(self)
        record = []
        while first or not sort:
            first = False
            sort = True
            for i in range(len(ll) - 1):
                if ll[i] > ll[i + 1]:
                    ll[i + 1],ll[i] = ll[i],ll[i + 1]
                    record.append(i + 1)
                    sort = False
                    break
        record.reverse()
        return record
    
    def from_code(cc):
        '''
        >>> c = [4, 1, 2, 0, 0]
        >>> Permutation.from_code(c)
        (5, 2, 4, 1, 3)
        '''
        ll =[i + 1 for i in range(len(cc))]
        result = []
        for val in cc:
            value = min(ll) if val == 0 else ll[val]
            result.append(value)
            ll.remove(value)
        return Permutation(result)
    
    def from_cycles(cc):
        '''
        >>> Permutation.from_cycles([[1, 5, 3, 4], [2]])
        (5, 2, 4, 1, 3)
        '''
        aux = {}
        for cycle in cc:
            for i in range(len(cycle) - 1):
                aux[cycle[i] - 1] = cycle[i + 1]
            aux[cycle[-1] - 1] = cycle[0]
        result = sorted(aux.items())
        return Permutation([item[1] for item in result])
    
    def from_reduced_dec(rd, d):
        '''
        >>> Permutation.from_reduced_dec([3, 4, 1, 2, 3, 2, 1],5)
        (5, 2, 4, 1, 3)
        '''
        ll = list(range(1, d + 1))
        for index in rd:
            ll[index],ll[index - 1] = ll[index - 1],ll[index]
        return Permutation(ll)
    
    def exc(self):
        '''
        >>> w = Permutation('58143762')
        >>> w.exc()
        3
        '''
        count = 0
        for i in range(1, len(self) + 1):
            if self(i) > i:
                count += 1
        return count
    
def standardization(w):
    '''
    >>> standardization('gagbga')
    (4, 1, 5, 3, 6, 2)
    >>> standardization([8,4,8,2,8,1])
    (4, 3, 5, 2, 6, 1)
    '''
    word = ''.join(sorted(str(v) for v in w))
    indexes = {}
    index = 1
    for i in range(len(word)):
        if word[i] not in indexes:
            indexes[word[i]] = [index]
        else:
            indexes[word[i]].append(index)
        index += 1
    return Permutation([indexes[str(letter)].pop(0) for letter in w])
        
    
def permlist(n):
    '''
    >>> permlist(3)
    [(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]
    '''
    def aux(result, ll):
        if ll == []:
            return [Permutation(result)]
        perms = []
        for value in ll:
            aa = list(ll)
            bb = list(result)
            aa.remove(value)
            bb.append(value)
            perms.extend(aux(bb, aa))
        return perms
    return aux([], list(range(1, n + 1)))

def Permutations(n):
    '''
    >>> S3 = Permutations(3)
    >>> perm = next(S3)
    >>> perm
    (1, 2, 3)
    >>> type(perm)
    <class 'permutations.Permutation'>
    '''
    return iter(permlist(n))

def genperms(ll):
    '''
    >>> it = genperms([1,3,5])
    >>> [next(it) for i in range(6)]
    [[1, 3, 5], [1, 5, 3], [3, 1, 5], [3, 5, 1], [5, 1, 3], [5, 3, 1]]
    '''
    return iter(listperms(ll))

def listperms(ll):
    '''
    >>> listperms([1,3,5])
    [[1, 3, 5], [1, 5, 3], [3, 1, 5], [3, 5, 1], [5, 1, 3], [5, 3, 1]]
    '''
    def aux(result, ll):
        if ll == []:
            return [result]
        perms = []
        for value in ll:
            aa = list(ll)
            bb = list(result)
            aa.remove(value)
            bb.append(value)
            perms.extend(aux(bb, aa))
        return perms
    return aux([], ll)

def gnome_sort(ll):
    '''
    >>> gnome_sort([5,4,3,2,1])
    ([1, 2, 3, 4, 5], [1, 2, 3, 4, 1, 2, 3, 1, 2, 1])
    '''
    sort = True
    first = True
    record = []
    while first or not sort:
        first = False
        sort = True
        for i in range(len(ll) - 1):
            if ll[i] > ll[i + 1]:
                ll[i + 1],ll[i] = ll[i],ll[i + 1]
                record.append(i + 1)
                sort = False
                break
    record.reverse()
    return ll, record

if __name__ == "__main__":
    import doctest
    doctest.testmod()

**********************************************************************
File "__main__", line 236, in __main__.Permutations
Failed example:
    type(perm)
Expected:
    <class 'permutations.Permutation'>
Got:
    <class '__main__.Permutation'>
**********************************************************************
1 items had failures:
   1 of   4 in __main__.Permutations
***Test Failed*** 1 failures.


In [2]:
from sympy import *

def show_coeff(expr, size):
    ll = []
    for i in range(size + 1):
        ll.append(expr.coeff(x, i))
    return ll

x, n = symbols('x n')
pascal = (x + 1)**n
for i in range(0, 14):
    print(show_coeff(pascal.subs(n, i).expand(), i))

[1]
[1, 1]
[1, 2, 1]
[1, 3, 3, 1]
[1, 4, 6, 4, 1]
[1, 5, 10, 10, 5, 1]
[1, 6, 15, 20, 15, 6, 1]
[1, 7, 21, 35, 35, 21, 7, 1]
[1, 8, 28, 56, 70, 56, 28, 8, 1]
[1, 9, 36, 84, 126, 126, 84, 36, 9, 1]
[1, 10, 45, 120, 210, 252, 210, 120, 45, 10, 1]
[1, 11, 55, 165, 330, 462, 462, 330, 165, 55, 11, 1]
[1, 12, 66, 220, 495, 792, 924, 792, 495, 220, 66, 12, 1]
[1, 13, 78, 286, 715, 1287, 1716, 1716, 1287, 715, 286, 78, 13, 1]


In [3]:
m = symbols('m')
expr = 1/(1 - x)**(-m)
for i in range(14):
    print(show_coeff(expr.subs(m, i).expand(), i))

[1]
[1, -1]
[1, -2, 1]
[1, -3, 3, -1]
[1, -4, 6, -4, 1]
[1, -5, 10, -10, 5, -1]
[1, -6, 15, -20, 15, -6, 1]
[1, -7, 21, -35, 35, -21, 7, -1]
[1, -8, 28, -56, 70, -56, 28, -8, 1]
[1, -9, 36, -84, 126, -126, 84, -36, 9, -1]
[1, -10, 45, -120, 210, -252, 210, -120, 45, -10, 1]
[1, -11, 55, -165, 330, -462, 462, -330, 165, -55, 11, -1]
[1, -12, 66, -220, 495, -792, 924, -792, 495, -220, 66, -12, 1]
[1, -13, 78, -286, 715, -1287, 1716, -1716, 1287, -715, 286, -78, 13, -1]


On remarque sur les coefficients ci-dessus qu'il y a aussi une similitude avec le triangle de Pascal

In [4]:
w = Permutation([5,2,4,1,3]); w

(5, 2, 4, 1, 3)

# Théorème de Foata

In [5]:
def foata(n):
    res = {}
    perms = permlist(n)
    for p in perms:
        perm = Permutation(p)
        desc = tuple(perm.descents())
        if desc not in res:
            res[desc] = [perm]
        else:
            res[desc].append(perm)
    return res


In [6]:
foata(4)

{(): [(1, 2, 3, 4)],
 (3,): [(1, 2, 4, 3), (1, 3, 4, 2), (2, 3, 4, 1)],
 (2,): [(1, 3, 2, 4), (1, 4, 2, 3), (2, 3, 1, 4), (2, 4, 1, 3), (3, 4, 1, 2)],
 (2, 3): [(1, 4, 3, 2), (2, 4, 3, 1), (3, 4, 2, 1)],
 (1,): [(2, 1, 3, 4), (3, 1, 2, 4), (4, 1, 2, 3)],
 (1, 3): [(2, 1, 4, 3),
  (3, 1, 4, 2),
  (3, 2, 4, 1),
  (4, 1, 3, 2),
  (4, 2, 3, 1)],
 (1, 2): [(3, 2, 1, 4), (4, 2, 1, 3), (4, 3, 1, 2)],
 (1, 2, 3): [(4, 3, 2, 1)]}

# Polynômes eulériens

In [10]:
def A(n, m):
    if n < m:
        return 0
    elif n + 1 == m or m == 0:
        return 1
    return (n - m) * A(n - 1, m - 1) + (m + 1) * A(n - 1, m)

In [11]:
for i in range(1, 10):
    for j in range(i + 1):
        value = A(i, j)
        if value != 0:
            print(value, end = ' ')
    print()

1 
1 1 
1 4 1 
1 11 11 1 
1 26 66 26 1 
1 57 302 302 57 1 
1 120 1191 2416 1191 120 1 
1 247 4293 15619 15619 4293 247 1 
1 502 14608 88234 156190 88234 14608 502 1 


# Nombre d'excédances

In [12]:
perms = Permutations(7)
count = {}
for p in perms:
    excedant = p.exc()
    if excedant not in count:
        count[excedant] = 1
    else:
        count[excedant] += 1
    print(p, p.exc())
print(count)

(1, 2, 3, 4, 5, 6, 7) 0
(1, 2, 3, 4, 5, 7, 6) 1
(1, 2, 3, 4, 6, 5, 7) 1
(1, 2, 3, 4, 6, 7, 5) 2
(1, 2, 3, 4, 7, 5, 6) 1
(1, 2, 3, 4, 7, 6, 5) 1
(1, 2, 3, 5, 4, 6, 7) 1
(1, 2, 3, 5, 4, 7, 6) 2
(1, 2, 3, 5, 6, 4, 7) 2
(1, 2, 3, 5, 6, 7, 4) 3
(1, 2, 3, 5, 7, 4, 6) 2
(1, 2, 3, 5, 7, 6, 4) 2
(1, 2, 3, 6, 4, 5, 7) 1
(1, 2, 3, 6, 4, 7, 5) 2
(1, 2, 3, 6, 5, 4, 7) 1
(1, 2, 3, 6, 5, 7, 4) 2
(1, 2, 3, 6, 7, 4, 5) 2
(1, 2, 3, 6, 7, 5, 4) 2
(1, 2, 3, 7, 4, 5, 6) 1
(1, 2, 3, 7, 4, 6, 5) 1
(1, 2, 3, 7, 5, 4, 6) 1
(1, 2, 3, 7, 5, 6, 4) 1
(1, 2, 3, 7, 6, 4, 5) 2
(1, 2, 3, 7, 6, 5, 4) 2
(1, 2, 4, 3, 5, 6, 7) 1
(1, 2, 4, 3, 5, 7, 6) 2
(1, 2, 4, 3, 6, 5, 7) 2
(1, 2, 4, 3, 6, 7, 5) 3
(1, 2, 4, 3, 7, 5, 6) 2
(1, 2, 4, 3, 7, 6, 5) 2
(1, 2, 4, 5, 3, 6, 7) 2
(1, 2, 4, 5, 3, 7, 6) 3
(1, 2, 4, 5, 6, 3, 7) 3
(1, 2, 4, 5, 6, 7, 3) 4
(1, 2, 4, 5, 7, 3, 6) 3
(1, 2, 4, 5, 7, 6, 3) 3
(1, 2, 4, 6, 3, 5, 7) 2
(1, 2, 4, 6, 3, 7, 5) 3
(1, 2, 4, 6, 5, 3, 7) 2
(1, 2, 4, 6, 5, 7, 3) 3
(1, 2, 4, 6, 7, 3, 5) 3
(1, 2, 4, 6, 7, 

(1, 5, 2, 7, 6, 4, 3) 3
(1, 5, 3, 2, 4, 6, 7) 1
(1, 5, 3, 2, 4, 7, 6) 2
(1, 5, 3, 2, 6, 4, 7) 2
(1, 5, 3, 2, 6, 7, 4) 3
(1, 5, 3, 2, 7, 4, 6) 2
(1, 5, 3, 2, 7, 6, 4) 2
(1, 5, 3, 4, 2, 6, 7) 1
(1, 5, 3, 4, 2, 7, 6) 2
(1, 5, 3, 4, 6, 2, 7) 2
(1, 5, 3, 4, 6, 7, 2) 3
(1, 5, 3, 4, 7, 2, 6) 2
(1, 5, 3, 4, 7, 6, 2) 2
(1, 5, 3, 6, 2, 4, 7) 2
(1, 5, 3, 6, 2, 7, 4) 3
(1, 5, 3, 6, 4, 2, 7) 2
(1, 5, 3, 6, 4, 7, 2) 3
(1, 5, 3, 6, 7, 2, 4) 3
(1, 5, 3, 6, 7, 4, 2) 3
(1, 5, 3, 7, 2, 4, 6) 2
(1, 5, 3, 7, 2, 6, 4) 2
(1, 5, 3, 7, 4, 2, 6) 2
(1, 5, 3, 7, 4, 6, 2) 2
(1, 5, 3, 7, 6, 2, 4) 3
(1, 5, 3, 7, 6, 4, 2) 3
(1, 5, 4, 2, 3, 6, 7) 2
(1, 5, 4, 2, 3, 7, 6) 3
(1, 5, 4, 2, 6, 3, 7) 3
(1, 5, 4, 2, 6, 7, 3) 4
(1, 5, 4, 2, 7, 3, 6) 3
(1, 5, 4, 2, 7, 6, 3) 3
(1, 5, 4, 3, 2, 6, 7) 2
(1, 5, 4, 3, 2, 7, 6) 3
(1, 5, 4, 3, 6, 2, 7) 3
(1, 5, 4, 3, 6, 7, 2) 4
(1, 5, 4, 3, 7, 2, 6) 3
(1, 5, 4, 3, 7, 6, 2) 3
(1, 5, 4, 6, 2, 3, 7) 3
(1, 5, 4, 6, 2, 7, 3) 4
(1, 5, 4, 6, 3, 2, 7) 3
(1, 5, 4, 6, 3, 7, 2) 4
(1, 5, 4, 6, 7, 

(3, 4, 2, 1, 5, 7, 6) 3
(3, 4, 2, 1, 6, 5, 7) 3
(3, 4, 2, 1, 6, 7, 5) 4
(3, 4, 2, 1, 7, 5, 6) 3
(3, 4, 2, 1, 7, 6, 5) 3
(3, 4, 2, 5, 1, 6, 7) 3
(3, 4, 2, 5, 1, 7, 6) 4
(3, 4, 2, 5, 6, 1, 7) 4
(3, 4, 2, 5, 6, 7, 1) 5
(3, 4, 2, 5, 7, 1, 6) 4
(3, 4, 2, 5, 7, 6, 1) 4
(3, 4, 2, 6, 1, 5, 7) 3
(3, 4, 2, 6, 1, 7, 5) 4
(3, 4, 2, 6, 5, 1, 7) 3
(3, 4, 2, 6, 5, 7, 1) 4
(3, 4, 2, 6, 7, 1, 5) 4
(3, 4, 2, 6, 7, 5, 1) 4
(3, 4, 2, 7, 1, 5, 6) 3
(3, 4, 2, 7, 1, 6, 5) 3
(3, 4, 2, 7, 5, 1, 6) 3
(3, 4, 2, 7, 5, 6, 1) 3
(3, 4, 2, 7, 6, 1, 5) 4
(3, 4, 2, 7, 6, 5, 1) 4
(3, 4, 5, 1, 2, 6, 7) 3
(3, 4, 5, 1, 2, 7, 6) 4
(3, 4, 5, 1, 6, 2, 7) 4
(3, 4, 5, 1, 6, 7, 2) 5
(3, 4, 5, 1, 7, 2, 6) 4
(3, 4, 5, 1, 7, 6, 2) 4
(3, 4, 5, 2, 1, 6, 7) 3
(3, 4, 5, 2, 1, 7, 6) 4
(3, 4, 5, 2, 6, 1, 7) 4
(3, 4, 5, 2, 6, 7, 1) 5
(3, 4, 5, 2, 7, 1, 6) 4
(3, 4, 5, 2, 7, 6, 1) 4
(3, 4, 5, 6, 1, 2, 7) 4
(3, 4, 5, 6, 1, 7, 2) 5
(3, 4, 5, 6, 2, 1, 7) 4
(3, 4, 5, 6, 2, 7, 1) 5
(3, 4, 5, 6, 7, 1, 2) 5
(3, 4, 5, 6, 7, 2, 1) 5
(3, 4, 5, 7, 1, 

(3, 7, 4, 1, 6, 2, 5) 4
(3, 7, 4, 1, 6, 5, 2) 4
(3, 7, 4, 2, 1, 5, 6) 3
(3, 7, 4, 2, 1, 6, 5) 3
(3, 7, 4, 2, 5, 1, 6) 3
(3, 7, 4, 2, 5, 6, 1) 3
(3, 7, 4, 2, 6, 1, 5) 4
(3, 7, 4, 2, 6, 5, 1) 4
(3, 7, 4, 5, 1, 2, 6) 4
(3, 7, 4, 5, 1, 6, 2) 4
(3, 7, 4, 5, 2, 1, 6) 4
(3, 7, 4, 5, 2, 6, 1) 4
(3, 7, 4, 5, 6, 1, 2) 5
(3, 7, 4, 5, 6, 2, 1) 5
(3, 7, 4, 6, 1, 2, 5) 4
(3, 7, 4, 6, 1, 5, 2) 4
(3, 7, 4, 6, 2, 1, 5) 4
(3, 7, 4, 6, 2, 5, 1) 4
(3, 7, 4, 6, 5, 1, 2) 4
(3, 7, 4, 6, 5, 2, 1) 4
(3, 7, 5, 1, 2, 4, 6) 3
(3, 7, 5, 1, 2, 6, 4) 3
(3, 7, 5, 1, 4, 2, 6) 3
(3, 7, 5, 1, 4, 6, 2) 3
(3, 7, 5, 1, 6, 2, 4) 4
(3, 7, 5, 1, 6, 4, 2) 4
(3, 7, 5, 2, 1, 4, 6) 3
(3, 7, 5, 2, 1, 6, 4) 3
(3, 7, 5, 2, 4, 1, 6) 3
(3, 7, 5, 2, 4, 6, 1) 3
(3, 7, 5, 2, 6, 1, 4) 4
(3, 7, 5, 2, 6, 4, 1) 4
(3, 7, 5, 4, 1, 2, 6) 3
(3, 7, 5, 4, 1, 6, 2) 3
(3, 7, 5, 4, 2, 1, 6) 3
(3, 7, 5, 4, 2, 6, 1) 3
(3, 7, 5, 4, 6, 1, 2) 4
(3, 7, 5, 4, 6, 2, 1) 4
(3, 7, 5, 6, 1, 2, 4) 4
(3, 7, 5, 6, 1, 4, 2) 4
(3, 7, 5, 6, 2, 1, 4) 4
(3, 7, 5, 6, 2, 

(4, 6, 1, 7, 5, 3, 2) 3
(4, 6, 2, 1, 3, 5, 7) 2
(4, 6, 2, 1, 3, 7, 5) 3
(4, 6, 2, 1, 5, 3, 7) 2
(4, 6, 2, 1, 5, 7, 3) 3
(4, 6, 2, 1, 7, 3, 5) 3
(4, 6, 2, 1, 7, 5, 3) 3
(4, 6, 2, 3, 1, 5, 7) 2
(4, 6, 2, 3, 1, 7, 5) 3
(4, 6, 2, 3, 5, 1, 7) 2
(4, 6, 2, 3, 5, 7, 1) 3
(4, 6, 2, 3, 7, 1, 5) 3
(4, 6, 2, 3, 7, 5, 1) 3
(4, 6, 2, 5, 1, 3, 7) 3
(4, 6, 2, 5, 1, 7, 3) 4
(4, 6, 2, 5, 3, 1, 7) 3
(4, 6, 2, 5, 3, 7, 1) 4
(4, 6, 2, 5, 7, 1, 3) 4
(4, 6, 2, 5, 7, 3, 1) 4
(4, 6, 2, 7, 1, 3, 5) 3
(4, 6, 2, 7, 1, 5, 3) 3
(4, 6, 2, 7, 3, 1, 5) 3
(4, 6, 2, 7, 3, 5, 1) 3
(4, 6, 2, 7, 5, 1, 3) 3
(4, 6, 2, 7, 5, 3, 1) 3
(4, 6, 3, 1, 2, 5, 7) 2
(4, 6, 3, 1, 2, 7, 5) 3
(4, 6, 3, 1, 5, 2, 7) 2
(4, 6, 3, 1, 5, 7, 2) 3
(4, 6, 3, 1, 7, 2, 5) 3
(4, 6, 3, 1, 7, 5, 2) 3
(4, 6, 3, 2, 1, 5, 7) 2
(4, 6, 3, 2, 1, 7, 5) 3
(4, 6, 3, 2, 5, 1, 7) 2
(4, 6, 3, 2, 5, 7, 1) 3
(4, 6, 3, 2, 7, 1, 5) 3
(4, 6, 3, 2, 7, 5, 1) 3
(4, 6, 3, 5, 1, 2, 7) 3
(4, 6, 3, 5, 1, 7, 2) 4
(4, 6, 3, 5, 2, 1, 7) 3
(4, 6, 3, 5, 2, 7, 1) 4
(4, 6, 3, 5, 7, 

(6, 4, 7, 3, 5, 1, 2) 3
(6, 4, 7, 3, 5, 2, 1) 3
(6, 4, 7, 5, 1, 2, 3) 4
(6, 4, 7, 5, 1, 3, 2) 4
(6, 4, 7, 5, 2, 1, 3) 4
(6, 4, 7, 5, 2, 3, 1) 4
(6, 4, 7, 5, 3, 1, 2) 4
(6, 4, 7, 5, 3, 2, 1) 4
(6, 5, 1, 2, 3, 4, 7) 2
(6, 5, 1, 2, 3, 7, 4) 3
(6, 5, 1, 2, 4, 3, 7) 2
(6, 5, 1, 2, 4, 7, 3) 3
(6, 5, 1, 2, 7, 3, 4) 3
(6, 5, 1, 2, 7, 4, 3) 3
(6, 5, 1, 3, 2, 4, 7) 2
(6, 5, 1, 3, 2, 7, 4) 3
(6, 5, 1, 3, 4, 2, 7) 2
(6, 5, 1, 3, 4, 7, 2) 3
(6, 5, 1, 3, 7, 2, 4) 3
(6, 5, 1, 3, 7, 4, 2) 3
(6, 5, 1, 4, 2, 3, 7) 2
(6, 5, 1, 4, 2, 7, 3) 3
(6, 5, 1, 4, 3, 2, 7) 2
(6, 5, 1, 4, 3, 7, 2) 3
(6, 5, 1, 4, 7, 2, 3) 3
(6, 5, 1, 4, 7, 3, 2) 3
(6, 5, 1, 7, 2, 3, 4) 3
(6, 5, 1, 7, 2, 4, 3) 3
(6, 5, 1, 7, 3, 2, 4) 3
(6, 5, 1, 7, 3, 4, 2) 3
(6, 5, 1, 7, 4, 2, 3) 3
(6, 5, 1, 7, 4, 3, 2) 3
(6, 5, 2, 1, 3, 4, 7) 2
(6, 5, 2, 1, 3, 7, 4) 3
(6, 5, 2, 1, 4, 3, 7) 2
(6, 5, 2, 1, 4, 7, 3) 3
(6, 5, 2, 1, 7, 3, 4) 3
(6, 5, 2, 1, 7, 4, 3) 3
(6, 5, 2, 3, 1, 4, 7) 2
(6, 5, 2, 3, 1, 7, 4) 3
(6, 5, 2, 3, 4, 1, 7) 2
(6, 5, 2, 3, 4, 

# Répartition du nombre d'excédance pour chaque permutation

On peut remarquer qu'il y a beaucoup de nombre d'excédance égal à 3 pour l'ensemble des permutations avec n = 8
On peut même voir que la répartition est égal entre 2 et 4 (avec 1191 permutations), entre 0 et 6 (avec une permutation) et entre 1 et 5 (avec 120 permutations)

# Foata

In [16]:
def rotate_maxi(ll):
    max_value = max(ll)
    while ll[0] != max_value:
        ll = ll[1:] + ll[:1]
    return ll

def foata1(perm):
    cycle = perm.cycles()
    res = []
    for c in cycle:
        res.append(rotate_maxi(c))
    res = sorted(res, key=lambda x: x[0])
    return Permutation(''.join([str(elt) for cycle in res for elt in cycle]))

In [17]:
foata1(Permutation('2314'))

(3, 1, 2, 4)

In [18]:
for p in Permutations(4): 
    q = foata1(p.inverse())
    print(p,q, p.exc(),q.des_num()) 

(1, 2, 3, 4) (1, 2, 3, 4) 0 0
(1, 2, 4, 3) (1, 2, 4, 3) 1 1
(1, 3, 2, 4) (1, 3, 2, 4) 1 1
(1, 3, 4, 2) (1, 4, 3, 2) 2 2
(1, 4, 2, 3) (1, 4, 2, 3) 1 1
(1, 4, 3, 2) (1, 3, 4, 2) 1 1
(2, 1, 3, 4) (2, 1, 3, 4) 1 1
(2, 1, 4, 3) (2, 1, 4, 3) 2 2
(2, 3, 1, 4) (3, 2, 1, 4) 2 2
(2, 3, 4, 1) (4, 3, 2, 1) 3 3
(2, 4, 1, 3) (4, 2, 1, 3) 2 2
(2, 4, 3, 1) (3, 4, 2, 1) 2 2
(3, 1, 2, 4) (3, 1, 2, 4) 1 1
(3, 1, 4, 2) (4, 3, 1, 2) 2 2
(3, 2, 1, 4) (2, 3, 1, 4) 1 1
(3, 2, 4, 1) (2, 4, 3, 1) 2 2
(3, 4, 1, 2) (3, 1, 4, 2) 2 2
(3, 4, 2, 1) (4, 2, 3, 1) 2 2
(4, 1, 2, 3) (4, 1, 2, 3) 1 1
(4, 1, 3, 2) (3, 4, 1, 2) 1 1
(4, 2, 1, 3) (2, 4, 1, 3) 1 1
(4, 2, 3, 1) (2, 3, 4, 1) 1 1
(4, 3, 1, 2) (4, 1, 3, 2) 2 2
(4, 3, 2, 1) (3, 2, 4, 1) 2 2
